In [30]:
import pickle
import os
import re

import numpy as np

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras import layers as L
from keras import backend as K

In [59]:
LE_CHANNEL = [
    np.array([1, 2, 4, 5]),
    np.array([1, 2, 4, 5])
]
RI_CHANNEL = [
    np.array([3, 2, 4]),
    np.array([1, 8, 9])
]
LABELS = np.array([1, 0])

In [3]:
PRE_EMBED = np.random.normal(size=(10, 8))

In [4]:
VOCAB_SIZE = 10
PRE_EMBED_SIZE = 8
TR_EMBED_SIZE = 8
BATCH_SIZE = 1
WINDOW_SIZE = 2
RNN_STATE_SIZE = 16

In [51]:
def object2vec_sentences():
    # INPUT CHANNELS
    # SHAPE_OUT: (batch, seq_len)
    
    le_channel = L.Input(
        shape=(None,),
        dtype='int32',
        name='LEFT_CHANNEL'
    )
    
    ri_channel = L.Input(
        shape=(None,),
        dtype='int32',
        name='RIGHT_CHANNEL'
    )
    
    # PRE_TRAINED EMBEDDINGS 

    pre_embeddings = L.Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=PRE_EMBED_SIZE,
        weights=[PRE_EMBED],
        trainable=False,
        name='PRE_EMBEDDINGS'
    )
    
    # CHANNELS EMBEDDINGS
    # SHAPE_OUT: (batch x seq_len x embed_size)
    
    le_embed = pre_embeddings(le_channel)
    ri_embed = pre_embeddings(ri_channel)
    
    # CHANNELS ENCODERS
    # SHAPE_OUT: (batch x rnn_state_size * 2)
    
    le_encoder = L.Bidirectional(
        layer=L.LSTM(RNN_STATE_SIZE),
        name='LEFT_ENCODER'
    )(le_embed)
    
    ri_encoder = L.Bidirectional(
        layer=L.LSTM(RNN_STATE_SIZE),
        name='RIGHT_ENCODER'
    )(ri_embed)
    
    # COMPARATOR:  HADAMARD PROD
    # SHAPE_OUT: (batch x rnn_state_size * 2)
    
    comparator = L.Lambda(lambda x: tf.multiply(*x))([le_encoder, ri_encoder])
    
    # DISCRIMINATOR
    discriminator = L.Dense(
        units=2,
        activation='softmax',
        name='DISCRIMINATOR'
    )(comparator)
    
    return Model(inputs=[le_channel, ri_channel], outputs=discriminator)

In [52]:
model = object2vec_sentences()

In [53]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [49]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
LEFT_CHANNEL (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
RIGHT_CHANNEL (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
PRE_EMBEDDINGS (Embedding)      (None, None, 8)      80          LEFT_CHANNEL[0][0]               
                                                                 RIGHT_CHANNEL[0][0]              
__________________________________________________________________________________________________
LEFT_ENCODER (Bidirectional)    (None, 32)           3200        PRE_EMBEDDINGS[0][0]             
__________

In [58]:
model.fit(x=[LE_CHANNEL, RI_CHANNEL], y=LABELS, epochs=10)

Epoch 1/10
2/2 [==============================] - 3s 1s/step - loss: 0.6903 - acc: 0.7500
Epoch 2/10
2/2 [==============================] - 0s 4ms/step - loss: 0.6894 - acc: 0.7500
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.6885 - acc: 0.7500
Epoch 4/10
2/2 [==============================] - 0s 4ms/step - loss: 0.6876 - acc: 0.7500
Epoch 5/10
2/2 [==============================] - 0s 4ms/step - loss: 0.6867 - acc: 0.7500
Epoch 6/10
2/2 [==============================] - 0s 12ms/step - loss: 0.6857 - acc: 0.7500
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 0.6847 - acc: 0.7500
Epoch 8/10
2/2 [==============================] - 0s 5ms/step - loss: 0.6838 - acc: 0.7500
Epoch 9/10
2/2 [==============================] - 0s 5ms/step - loss: 0.6828 - acc: 0.7500
Epoch 10/10
2/2 [==============================] - 0s 10ms/step - loss: 0.6817 - acc: 1.0000


In [47]:
r = model.predict(x=[LE_CHANNEL[:2], RI_CHANNEL[:2]])

In [48]:
r.shape

(2, 2)

In [189]:
def object2vec_words():
    input_context = L.Input(
        shape=(WINDOW_SIZE,),
        dtype='int32',
        name='INPUT_CONTEXTS'
    )

    input_target = L.Input(
        shape=(1,),
        dtype='int32',
        name='INPUT_LABELS'
    )

    # PRE_TRAINED EMBEDDINGS 

    pre_embed = L.Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=PRE_EMBED_SIZE,
        weights=[corpus.W2V_EMBED],
        trainable=False,
        name='PRE_EMBEDDINGS'
    )(input_context)

    # TRAINABLE EMBEDDINGS 

    tr_embed = L.Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=TR_EMBED_SIZE,
        name='TRAINABLE_EMBEDDINGS'
    )(input_target)

    # COLLAPSE TRAINABLE EMBEDDINGS TO SHAPE: [batch_size x TR_EMBED_SIZE]

    tr_embed = L.Reshape(
        target_shape=(TR_EMBED_SIZE,),
        name='COLLAPSED_TRAINABLE_EMBEDDINGS'
    )(tr_embed)

    # CONCAT TRAINABLE AND PRE-TRAINED EMBEDDINGS WITHIN ENCODER 

    tr_embed_to_encoder = L.Lambda(
        lambda x: K.repeat(x, WINDOW_SIZE),
        name='TRAINABLE_EMBEDDINGS_LAMBDA'
    )(tr_embed) 

    encoder_input = L.Concatenate(name='ENCODER_INPUT')([pre_embed, tr_embed_to_encoder])

    # ENCODING CONTEXT

    encoder = L.Bidirectional(
        layer=L.CuDNNLSTM(RNN_STATE_SIZE),
        name='ENCODER'
    )(encoder_input)

    # MERGING ENCODED CONTEXT WITH TARGET EMBEDDINGS
    # SHAPE: [batch_size x RNN_STATE_SIZE * 2 + TR_EMBED_SIZE]

    comparator = L.Concatenate(name='COMPARATOR')([encoder, tr_embed])

    # DENSE

    dense_1 = L.Dense(
        units=128,
        activation='relu',
        name='DENSE_1'
    )(comparator)

    # OUTPUT PREDICTIONS

    dense_out = L.Dense(
        units=2,
        activation='softmax',
        name='DENSE_OUT'
    )(dense_1)
    
    return Model(inputs=[input_context, input_target], outputs=dense_out)

In [ ]:
def w2v_fine_tune():
    input_context = L.Input(
        shape=(WINDOW_SIZE,),
        dtype='int32',
        name='INPUT_CONTEXTS'
    )

    input_target = L.Input(
        shape=(1,),
        dtype='int32',
        name='INPUT_LABELS'
    )

    # PRE_TRAINED EMBEDDINGS 

    embedding = L.Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=EMBED_SIZE,
        weights=[corpus.W2V_EMBED],
        trainable=True,
        name='PRE_EMBEDDINGS'
    )

    context_embed = embedding(input_context)

    # TRAINABLE EMBEDDINGS 

    target_embed = embedding(input_target)

    # COLLAPSE TRAINABLE EMBEDDINGS TO SHAPE: [batch_size x TR_EMBED_SIZE]

    target_embed = L.Reshape(
        target_shape=(EMBED_SIZE,),
        name='COLLAPSED_TRAINABLE_EMBEDDINGS'
    )(target_embed)

    # ENCODING CONTEXT

    encoder = L.Bidirectional(
        layer=L.CuDNNLSTM(RNN_STATE_SIZE),
        name='ENCODER'
    )(context_embed)

    # MERGING ENCODED CONTEXT WITH TARGET EMBEDDINGS
    # SHAPE: [batch_size x RNN_STATE_SIZE * 2 + TR_EMBED_SIZE]

    comparator = L.Dot(axes=-1, name='COMPARATOR')([encoder, target_embed])

    # DENSE

    # dense_1 = Dense(
    #     units=128,
    #     activation='relu',
    #     name='DENSE_1'
    # )(comparator)

    # OUTPUT PREDICTIONS

    dense_out = L.Dense(
        units=2,
        activation='softmax',
        name='DENSE_OUT'
    )(comparator)
    
    return Model(inputs=[input_context, input_target], outputs=dense_out)

In [190]:
model = Model(inputs=[input_context, input_labels], outputs=dense)

In [191]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [175]:
pre_embed.shape

TensorShape([Dimension(None), Dimension(2), Dimension(8)])

In [163]:
model.fit(x=[CONTEXTS_DATA[:100], LABELS_DATA[:100]], y=LABELS_DATA[:100], epochs=10, batch_size=8, verbose=True)

Epoch 1/10
100/100 [==============================] - 1s 13ms/step - loss: 10.3144 - acc: 0.2200
Epoch 2/10
100/100 [==============================] - 1s 13ms/step - loss: 10.2804 - acc: 0.3700
Epoch 3/10
100/100 [==============================] - 1s 13ms/step - loss: 10.2388 - acc: 0.4300
Epoch 4/10
100/100 [==============================] - 1s 13ms/step - loss: 10.1880 - acc: 0.5100
Epoch 5/10
100/100 [==============================] - 1s 13ms/step - loss: 10.1179 - acc: 0.5200
Epoch 6/10
100/100 [==============================] - 1s 13ms/step - loss: 10.0243 - acc: 0.4700
Epoch 7/10
100/100 [==============================] - 1s 13ms/step - loss: 9.8916 - acc: 0.4200
Epoch 8/10
100/100 [==============================] - 1s 13ms/step - loss: 9.7167 - acc: 0.3400
Epoch 9/10
100/100 [==============================] - 1s 13ms/step - loss: 9.4753 - acc: 0.3100
Epoch 10/10
100/100 [==============================] - 1s 13ms/step - loss: 9.1582 - acc: 0.2600


In [160]:
r = model.predict([CONTEXTS_DATA[:5], LABELS_DATA[:5]])

In [161]:
r

array([[3.2178265e-05, 3.1872823e-05, 3.1878761e-05, ..., 3.2254604e-05,
        3.2435961e-05, 3.2233638e-05],
       [3.1995096e-05, 3.1848977e-05, 3.2101099e-05, ..., 3.2322718e-05,
        3.2207776e-05, 3.2370219e-05],
       [3.2093802e-05, 3.2129152e-05, 3.1940173e-05, ..., 3.2114731e-05,
        3.2211134e-05, 3.1697680e-05],
       [3.1949268e-05, 3.1947879e-05, 3.1997937e-05, ..., 3.2049498e-05,
        3.2346343e-05, 3.2284624e-05],
       [3.2119133e-05, 3.2424490e-05, 3.1950032e-05, ..., 3.1951193e-05,
        3.2050219e-05, 3.1822066e-05]], dtype=float32)

In [101]:
tf.reset_default_graph()
tf_sess.close()

In [102]:
# window_left + window_right
tf_vocab_size = 9
tf_fitting_embed_size = 8
tf_batch_size = 1
tf_window_size = 2
tf_rnn_state_size = 16
tf_embedding_dense_size = 32
tf_pre_embedding = tf.constant(pre_embedding, dtype=tf.float32)

tf_input_context = tf.placeholder(dtype=tf.int32, shape=(tf_batch_size, tf_window_size))
tf_input_labels = tf.placeholder(dtype=tf.int32, shape=(tf_batch_size,))

tf_fitting_embedding = tf.Variable(tf.truncated_normal(shape=(tf_vocab_size, tf_fitting_embed_size), stddev=0.1))
tf_W_out = tf.Variable(tf.truncated_normal(shape=(tf_rnn_state_size * 2 + tf_fitting_embed_size, tf_vocab_size), stddev=0.1))
tf_b_out = tf.Variable(tf.zeros(shape=(tf_vocab_size,)))


# EMBEDDING TENSOR [batch_size x window_size x pre_embed_size]

with tf.device("/cpu:0"):
    tf_context_pre_embed = tf.nn.embedding_lookup(tf_pre_embedding, tf_input_context)

tf_rnn_cell_fw = tf.nn.rnn_cell.LSTMCell(16)
tf_rnn_cell_bw = tf.nn.rnn_cell.LSTMCell(16)
tf_rnn_state = tf_rnn_cell.zero_state(tf_batch_size, dtype=tf.float32)

tf_rnn_outputs, tf_rnn_states = tf.nn.bidirectional_dynamic_rnn(
    dtype=tf.float32,
    cell_fw=tf_rnn_cell_fw,
    cell_bw=tf_rnn_cell_bw,
    inputs=tf_context_pre_embed
)

with tf.device("/cpu:0"):
    tf_context_fitting_embed = tf.nn.embedding_lookup(tf_fitting_embedding, tf_input_labels)

# CONCAT FW AND BW HIDDEN STATES AND FITTING EMBED OF TARGETS [batch_size x HIDDEN_STATE_SIZE * 2 + FITTING_EMBED_SIZE]

tf_final_state = tf.concat(
    [tf_rnn_states[0].c, tf_rnn_states[1].c, tf_context_fitting_embed],
    axis=1
)

tf_logits = tf.matmul(tf_final_state, tf_W_out) + tf_b_out

tf_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=tf_input_labels,
    logits=tf_logits
)

In [103]:
tf_init_op = tf.global_variables_initializer()
tf_sess = tf.Session()
tf_sess.run(tf_init_op)

In [104]:
r = tf_sess.run(
        tf_final_state,
        {tf_input_context: np.array([[0, 2]]), tf_input_labels: np.array([1])}
)

In [105]:
r.shape

(1, 40)

In [40]:
r.c

array([[ 0.05075238, -0.2744412 , -0.23570031, -0.01703531, -0.00418064,
         0.34944135,  0.30708185, -0.15840204,  0.632811  , -0.20224315,
         0.20672935, -0.3125187 ,  0.16155767,  0.38695666, -0.1943923 ,
         0.20426655]], dtype=float32)

In [47]:
len(embeddings)

9

In [72]:
tf.concat(
    [tf.constant(np.array([[1]])), tf.constant(np.array([[2]]))], 1
).eval(session=tf_sess)

array([[1, 2]])